## Frequency of pupillometry measurements

In [ ]:
import pandas as pd
from utils import load_encrypted_xlsx
import os

In [ ]:
registry_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/post_hoc_modified_aSAH_DATA_2009_2023_24122023.xlsx'
pupillometry_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/pdms_data/Transfer Urs.pietsch@kssg.ch 22.01.24, 15_34/20240117_SAH_SOS_Pupillometrie.csv'
registry_pdms_correspondence_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/pdms_data/registry_pdms_correspondence.csv'
output_path = '/Users/jk1/Downloads'

In [ ]:
registry_df = load_encrypted_xlsx(registry_data_path)
pupillometry_df = pd.read_csv(pupillometry_data_path, sep=';', decimal='.')
registry_pdms_correspondence_df = pd.read_csv(registry_pdms_correspondence_path)

In [ ]:
registry_pdms_correspondence_df.rename(columns={'JoinedName': 'Name'}, inplace=True)
registry_pdms_correspondence_df['Date_birth'] = pd.to_datetime(registry_pdms_correspondence_df['Date_birth'], format='%Y-%m-%d')
registry_df = registry_df.merge(registry_pdms_correspondence_df, on=['SOS-CENTER-YEAR-NO.', 'Name', 'Date_birth'], how='left')

In [ ]:
registry_df.Date_Ictus.fillna(registry_df.Date_admission, inplace=True)
pupillometry_df = pupillometry_df.merge(registry_df[['pNr', 'Date_Ictus']], on='pNr', how='left')

## Frequency of measurements

In [ ]:
# for every pNr get the frequency of measurements (timings in timePupil)
pupillometry_df['timePupil'] = pd.to_datetime(pupillometry_df['timePupil'])
# groupby pNr; then sort by timePupil and get the difference between consecutive timePupil values
pupillometry_df = pupillometry_df.sort_values('timePupil')
pupillometry_df['timePupilDiff'] = pupillometry_df.groupby('pNr')['timePupil'].diff()
# convert to hours
pupillometry_df['timePupilDiff'] = pupillometry_df['timePupilDiff'].dt.total_seconds() / (60*60)

In [ ]:
pupillometry_df.drop_duplicates(subset=['pNr', 'timePupil'], inplace=True)

In [ ]:
pupillometry_df['timePupilDiff'].describe()

### Plot Time between measurements over time

In [ ]:
pupillometry_df['relative_time'] = pd.to_datetime(pupillometry_df['timePupil']) - pd.to_datetime(pupillometry_df['Date_Ictus'])
pupillometry_df['relative_time'] = pupillometry_df['relative_time'].dt.total_seconds() / (60*60*24)
pupillometry_df.loc[~pupillometry_df['relative_time'].isna(), 'relative_time_cat'] = pupillometry_df[~pupillometry_df['relative_time'].isna()]['relative_time'].astype(int)
pupillometry_df.loc[~pupillometry_df['timePupilDiff'].isna(), 'timePupilDiff_cat'] = pupillometry_df[~pupillometry_df['timePupilDiff'].isna()]['timePupilDiff'].astype(int)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

temp_df = pupillometry_df[pupillometry_df['timePupilDiff'] < 20]

ax = sns.scatterplot(data=temp_df, x='relative_time', y='timePupilDiff', hue='timePupilDiff', alpha=0.5, palette='rocket')
ax.set(xlabel='Time since event (days)', ylabel='Time between measurements (hours)')

In [ ]:
fg = sns.displot(data=pupillometry_df, x='relative_time_cat', y='timePupilDiff_cat', kind='kde', fill=True, cmap='rocket', cbar=True, cbar_kws={'label': 'Density of measurements'}, 
                 cut=0)
fg.ax.set(xlabel='Time since event (days)', ylabel='Time between measurements (hours)')
fg.ax.set_ylim(0, 20)

In [ ]:
# fg.savefig(os.path.join(output_path, 'pupillometry_frequency.tiff'), dpi=300)